In [1]:
from transformers.models.clip.modeling_clip import CLIPTextModel
from diffusers import DDIMScheduler
from multi_token_clip import MultiTokenCLIPTokenizer
from diffusers import StableDiffusionPipeline
import torch

In [2]:
# 将 model_path 设置为 Stable-Diffusion-v1.5 的模型路径或名字
model_or_name = "../../model/"

text_encoder = CLIPTextModel.from_pretrained(model_or_name, subfolder="text_encoder", revision=False)
tokenizer = MultiTokenCLIPTokenizer.from_pretrained(model_or_name, subfolder="tokenizer")
scheduler = DDIMScheduler(
    beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", 
    clip_sample=False, set_alpha_to_one=False
)
model = StableDiffusionPipeline.from_pretrained(
    model_or_name, scheduler=scheduler, 
    text_encoder=text_encoder, tokenizer=tokenizer
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CLIPTokenizer'. 
The class this function is called from is 'MultiTokenCLIPTokenizer'.
D:\anaconda\lib\site-packages\diffusers\pipelines\stable_diffusion\pipeline_stable_diffusion.py:101: FutureWarning: The configuration file of this scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.10.1",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "set_alpha_to_one": false,
  "steps_offset": 0,
  "trained_betas": null
}
 is outdated. `steps_offset` should be set to 1 instead of 0. Please make sure to update the config accordingly as leaving `steps_offset` might led to incorrect results in future versions. If you have downloaded this

In [3]:
# 设置一些路径和参数
from inversion_free import gen_inversion_free
from utils import img2latent, latent2img
from datetime import datetime
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载图片
# 第一级目录
img_dir_src_1 = "./pair/"
img_dir_tar_1 = img_dir_src_1
# 第二级目录
img_dir_src_2 = "horse"
src_img_dir = os.path.join(img_dir_src_1, img_dir_src_2)
tar_img_dir_2 = "zebra"
tar_img_dir = os.path.join(img_dir_tar_1, tar_img_dir_2)

def get_image_file(path):
    img_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')
    img_files = [f for f in os.listdir(path) if f.lower().endswith(img_extensions)]
    return img_files[0]

src_img_file = get_image_file(src_img_dir)
tar_img_file = get_image_file(tar_img_dir)

src_latent = img2latent(os.path.join(src_img_dir, src_img_file), model)
tar_latent = img2latent(os.path.join(tar_img_dir, tar_img_file), model)

# 加载嵌入向量
# 第一级目录
embed_dir_src_1 = "./output/"
embed_dir_tar_1 = embed_dir_src_1

# 第二级目录
embed_dir_src_2 = "hrs"
embed_dir_src_2 = os.path.join(embed_dir_src_1, embed_dir_src_2)
tar_embed_dir_2 = "zbr"
tar_embed_dir_2 = os.path.join(embed_dir_tar_1, tar_embed_dir_2)

# 嵌入向量名字
src_embed_dir = "horse_04_25_2024_1603"
src_embed_dir = os.path.join(embed_dir_src_2, src_embed_dir)
tar_embed_dir = "zebra_04_25_2024_1557"
tar_embed_dir = os.path.join(tar_embed_dir_2, tar_embed_dir)
# 嵌入向量训练步数
src_steps = 100
tar_steps = src_steps

src_embedding = torch.load(os.path.join(src_embed_dir, f"{src_steps}.bin"))
tar_embedding = torch.load(os.path.join(tar_embed_dir, f"{tar_steps}.bin"))

In [4]:
# 设置输出目录并生成
date = datetime.now().strftime("%Y-%m-%d")
output_dir = f"./output_img/{str(date)}-{img_dir_src_2}-to-{tar_img_dir_2}/"
os.makedirs(output_dir, exist_ok=True)

time = datetime.now().strftime("%H-%M-%S")
num_inference_steps = 20
save_all = False

# 选择权重的类型
inclination = "none-src"
mode = "cosine"
cfg_guidance_scale = 0.7

# src 和 tar 方向的系数
src_coefficient = 1
tar_coefficient = 1

output_img_name = f"{time}_{src_img_file[:-4]}2{tar_img_file[:-4]}_{num_inference_steps}steps_{inclination}_{mode[:3]}_cfg{cfg_guidance_scale}_src{src_coefficient}_tar{tar_coefficient}"

# 生成图片
latents = gen_inversion_free(
    model.to(device), src_latent, tar_latent, src_embedding, tar_embedding,
    num_inference_steps=num_inference_steps, mode=mode, inclination=inclination,
    cfg_guidance=cfg_guidance_scale, src_coef=src_coefficient, tar_coef=tar_coefficient,
)


if save_all:
    for i, latent in enumerate(latents):
        img = latent2img(latent.detach(), model)
        print(f"Saving {output_img_name}_{i}.png in {output_dir}")
        img.save(os.path.join(output_dir, f"{output_img_name}_{i}.png"))
else:
    img = latent2img(latents[-1].detach(), model)
    print(f"Saving {output_img_name}.png in {output_dir}")
    img.save(os.path.join(output_dir, f"{output_img_name}.png"))

100%|██████████| 19/19 [07:17<00:00, 23.05s/it]


Saving 02-52-42_horse2zebra_20steps_none-src_cos_cfg0.7_src1_tar1.png in ./output_img/2024-05-10-horse-to-zebra/
